# **VIX Backtesting Around Fed Rate Announcements**
This project analyzes whether buying the VIX the day before Federal Reserve (FOMC) rate announcements and selling it at the close of announcement day is historically profitable.

---

## **Executive Summary**
- **Objective:** Test if anticipated volatility around FOMC announcements offers a tradable edge in VIX.
- **Approach:** Scrape FOMC meeting dates, fetch VIX data, simulate trades (T-1 buy → T sell), and calculate performance metrics.
- **Key Findings:**
  - Win rate: historically poor
  - Max drawdown: extremely high
  - Cumulative loss: -99.43%

Conclusion: **This naive volatility timing strategy is not robust and would have destroyed capital.**


## **Background**
Stocks plunged on Wednesday, 12/18/24 after Fed Chair Jerome Powell's inflationary guidance that accompanied a 25 bps rate cut. My friend told me that he bought 2x leveraged VIX shares having anticipated that "no matter what the announcement was", volatility would be significant and VIX would move up. Indeed, VIX soared ~50% that day. I was curious to see if this would actually work historically.


## **Step 1: Data Acquisition (VIX Prices)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Download daily VIX data
vix_data = yf.download("^VIX", start="1990-01-01", end="2024-12-19", interval="1d")
vix_daily = vix_data[['Open','Close']].reset_index()
vix_daily.head()

## **Step 2: Scrape FOMC Meeting Dates**
We extract all scheduled FOMC rate decision dates (excluding unscheduled/emergency meetings).

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

fed_dates = []

def extract_fomc_second_dates(html):
    soup = BeautifulSoup(html,'html.parser')
    elements = soup.find_all('h5', class_='panel-heading panel-heading--shaded')
    legacy_elements = soup.select('div.panel-heading > h5')
    elements.extend(legacy_elements)

    for el in elements:
        text = el.get_text(strip=True)
        parts = text.split(' ')
        if len(parts) < 3 or "(unscheduled)" in text or "Call" in text:
            continue
        try:
            date_str = ' '.join(parts[-3:]).replace(',','')
            date = datetime.strptime(date_str, "%B %d %Y").strftime('%Y-%m-%d')
            fed_dates.append(date)
        except:
            continue

# Scrape historical pages (1990–2018)
for yr in range(1990,2019):
    url = f'https://www.federalreserve.gov/monetarypolicy/fomchistorical{yr}.htm'
    html = requests.get(url).text
    extract_fomc_second_dates(html)

# Scrape modern calendar (2019+)
def scrape_fomc_dates_new(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    for section in soup.find_all('div', class_='panel panel-default'):
        year_text = section.find('div', class_='panel-heading').get_text(strip=True)
        year = year_text.split(' ')[0]
        for row in section.select('tr.shadedrow, tr'):
            cells = row.find_all('td')
            if not cells: continue
            date_text = cells[0].get_text(strip=True)
            if "unscheduled" in date_text.lower(): continue
            try:
                date = datetime.strptime(date_text,"%B %d, %Y").strftime('%Y-%m-%d')
                fed_dates.append(date)
            except:
                continue

scrape_fomc_dates_new("https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm")
len(fed_dates)

## **Step 3: Backtesting Logic**
We simulate:
- **Buy:** VIX at close of day before FOMC date  
- **Sell:** VIX at close of FOMC date  

We then compute % gains, win rate, drawdowns, and cumulative portfolio return.

In [ ]:
vix_daily['Date'] = pd.to_datetime(vix_daily['Date'])
fed_dates = pd.to_datetime(fed_dates)
t_minus_1 = [d - pd.Timedelta(days=1) for d in fed_dates]

trades = []
for i, fed_date in enumerate(fed_dates):
    day_before = vix_daily[vix_daily['Date'] == t_minus_1[i]]
    fed_day = vix_daily[vix_daily['Date'] == fed_date]
    if day_before.empty or fed_day.empty:
        continue
    t1_close = day_before.iloc[0]['Close']
    t_close = fed_day.iloc[0]['Close']
    profit_pct = (t_close - t1_close) / t1_close * 100
    trades.append({'Fed Date':fed_date,'t-1 Close':t1_close,'t Close':t_close,'Profit %':profit_pct})

results = pd.DataFrame(trades)
results.head()

In [ ]:
import matplotlib.pyplot as plt

# Metrics
win_rate = (results['Profit %']>0).mean()*100
max_drawdown = results['Profit %'].min()
max_win = results['Profit %'].max()
initial_value=10000
portfolio=initial_value
for p in results['Profit %']:
    portfolio *= (1+p/100)
cumulative_return=(portfolio/initial_value-1)*100

print(f"Win Rate: {win_rate:.2f}%")
print(f"Max Drawdown: {max_drawdown:.2f}%")
print(f"Max Win: {max_win:.2f}%")
print(f"Cumulative Return: {cumulative_return:.2f}% (Final Value: ${portfolio:.2f})")

# Plot distribution of returns
plt.hist(results['Profit %'], bins=20, edgecolor='black')
plt.title("Distribution of VIX Strategy Returns")
plt.xlabel("Profit %")
plt.ylabel("Frequency")
plt.show()

## **Conclusion**
- Strategy win rate is poor, cumulative returns are catastrophic (-99.43%).
- Anticipating volatility by buying VIX pre-FOMC announcements is not robust.
- Further enhancements could involve:
  - Implied vs realized volatility spreads
  - Options backtesting (e.g., straddles pre-announcement)
  - Using intraday data, which I could not access

